In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestRegressor

In [2]:
df_yelp = pd.read_parquet("D:\\Desktop\\proyecto_grupal\\sprint1\\df_yelp.parquet")
df_atributos = pd.read_parquet("D:\\Desktop\\proyecto_grupal\\sprint1\\df_atributos.parquet")

In [6]:
# Contamos la frecuencia de cada ciudad
city_counts = df_yelp['city'].value_counts()

# Seleccionamos las 5 ciudades más frecuentes
top_cities = city_counts.nlargest(5).index

# Filtramos el DataFrame para que solo tenga las 5 ciudades principales
df_filtered = df_yelp[df_yelp['city'].isin(top_cities)]

# Ahora, aplicamos OneHotEncoder solo a las ciudades seleccionadas
encoder = OneHotEncoder(sparse_output=False)
city_encoded = encoder.fit_transform(df_filtered[['city']])

# Convertimos el resultado en un DataFrame con las columnas correspondientes
city_encoded_df = pd.DataFrame(city_encoded, columns=encoder.categories_[0])

# Unimos las columnas codificadas al DataFrame original
df_filtered = pd.concat([df_filtered, city_encoded_df], axis=1)

# Crear DataFrame
df = pd.DataFrame(df_filtered)

df_filtered_selected = df_filtered[['name', 'city', 'review_count', 'funny']]
df_atributos_selected = df_atributos[["RestaurantsGoodForGroups","BusinessAcceptsCreditCards","GoodForKids","Smoking","BusinessParking"]]

# Preprocesamiento de datos
preprocessor = ColumnTransformer(
    transformers=[
        ('city', OneHotEncoder(), ['city']),
        ('num', 'passthrough', ['review_count', 'funny', 'RestaurantsGoodForGroups',
       'BusinessAcceptsCreditCards', 'GoodForKids', 'Smoking',
       'BusinessParking'])
    ])

# Crear el modelo de regresión
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

# Concatenamos las dos partes a lo largo de las columnas (axis=1)
X = pd.concat([df_filtered_selected, df_atributos_selected], axis=1)

y = df_filtered['stars_y']

X = X.dropna(subset=["name"	,"city"	,"review_count"	,"funny"])

y = y[X.index]

# Entrenamiento del modelo
model.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('city', OneHotEncoder(),
                                                  ['city']),
                                                 ('num', 'passthrough',
                                                  ['review_count', 'funny',
                                                   'RestaurantsGoodForGroups',
                                                   'BusinessAcceptsCreditCards',
                                                   'GoodForKids', 'Smoking',
                                                   'BusinessParking'])])),
                ('regressor', RandomForestRegressor())])

In [7]:
# Función para predecir el restaurante más cercano según las características
def predecir_restaurante(model, caracteristicas):
    # Eliminar 'nombre_restaurante' de las características para la predicción
    caracteristicas = {k: v for k, v in caracteristicas.items() if k != 'name'}
    
    # Transformar las características a la forma que el modelo espera
    caracteristicas_df = pd.DataFrame([caracteristicas])
    
    # Preprocesar las características (aplicar el mismo preprocesamiento al nuevo dato)
    X_transformed = model.named_steps['preprocessor'].transform(caracteristicas_df)
    
    # Usar similitudes de coseno para encontrar el restaurante más cercano
    X_full_transformed = model.named_steps['preprocessor'].transform(X)  # Datos completos transformados
    cos_similarities = cosine_similarity(X_full_transformed, X_transformed)
    
    # Encontrar el índice del restaurante más similar
    idx_mas_similar = np.argmax(cos_similarities)
    
    # Retornar el restaurante más similar
    restaurante_similar = df.iloc[idx_mas_similar]
    return restaurante_similar['name'], restaurante_similar['stars_y']

In [8]:
# Características del restaurante deseado
caracteristicas_deseadas = {
    'city':"Indianapolis", 
    'review_count':89, 
    'funny':0, 
    'RestaurantsGoodForGroups':0,
    'BusinessAcceptsCreditCards':1,
    'GoodForKids':1,
    'Smoking':0,
    'BusinessParking': 1
}

# Predecir el restaurante más cercano
restaurante, estrellas = predecir_restaurante(model, caracteristicas_deseadas)
print(f"El restaurante más cercano es: {restaurante} con una calificación predicha de {estrellas}")

El restaurante más cercano es: Pie Five Pizza Company con una calificación predicha de 2.0
